## Math 157: Intro to Mathematical Software
## UC San Diego, winter 2018

## Homework 4: due February 9, 2018

Please enter all answers within this notebook unless otherwise specified. As usual, don't forget to cite sources and collaborators.

Through this problem set, use the SageMath 8.1 kernel.

### Problem 1: QR codes

Grading criterion: correctness of code and output.

1a. The following image is the file "qr.gif" in this folder: ![qr.gif](qr.gif). Using the command `scipy.misc.imread`, convert this QR code into the underlying 0-1 matrix that it represents (black = 0, white = 1). You might want to read the [Scipy Lectures section on image processing](http://www.scipy-lectures.org/advanced/image_processing/) for some context on image processing.




In [154]:
import scipy.misc
import numpy as np

mat = scipy.misc.imread('qr.gif')
mat = np.array(mat)
mat[mat==255] = 1
mat[mat==0] = 0

1b. Using the command `matrix_plot`, convert your answer for part (a) back into a QR code. Your answer should be legible to a standard QR code reader (e.g., on your phone).

In [155]:
matrix_plot(mat)

### Problem 2: Timing

Grading criterion: correctness of code.

2a. Write a straightforward Python function (that is, directly implement the definition without using any Sage or NumPy shortcuts) to compute the standard deviation of a list of numbers.

In [18]:
def f(v):
    u = [(x-mean(v))^2 for x in v]
    return sqrt(sum(u)/(len(u))) #formula for population standard deviation

2b. Using `timeit`, compare the speed of your program when given as input `v = range(1000)`,  `v = range(10000)` and `v = range(100000)` with the built-in functions provided by the following systems:
- numpy;
- R (use `r.???` to access R functions);
- `stats.TimeSeries`.

In [19]:
import numpy as np
v = np.array(range(1000))
timeit('f(v)')
timeit('np.std(v)')
timeit('r.sd(v)')
timeit('stats.TimeSeries.standard_deviation(finance.TimeSeries(v))')

25 loops, best of 3: 27.9 ms per loop
625 loops, best of 3: 31.2 µs per loop


25 loops, best of 3: 24.3 ms per loop
625 loops, best of 3: 4.68 µs per loop


In [8]:
import numpy as np
v = np.array(range(10000))
timeit('f(v)')
timeit('np.std(v)')
timeit('r.sd(v)')
timeit('stats.TimeSeries.standard_deviation(finance.TimeSeries(v))')

5 loops, best of 3: 482 ms per loop


625 loops, best of 3: 67.3 µs per loop


25 loops, best of 3: 16.3 ms per loop
625 loops, best of 3: 39.6 µs per loop


In [22]:
v = np.array(range(100000))
timeit('f(v)',number=1,repeat=1)
timeit('np.std(v)')
timeit('r.sd(v)')
timeit('stats.TimeSeries.standard_deviation(finance.TimeSeries(v))')

1 loops, best of 1: 18.1 s per loop


625 loops, best of 3: 355 µs per loop


25 loops, best of 3: 15.7 ms per loop


625 loops, best of 3: 374 µs per loop


### Problem 3: Playing cards

Grading criterion: correctness of code and output.

Overall hint: you may want to read this [tutorial](https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/tutorial.html) which solves an easier version of this problem.

3a. Form a "deck of playing cards" by making a list of 52 tuples, each of which is an ordered pair whose first member is one of the 13 possible ranks (A,2,3,4,5,6,7,8,9,T,J,Q,K) and whose second member is one of the 4 possible suits (C,D,H,S).

In [41]:
#suits = Set(['C','D','H','S'])
#ranks = Set(["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K","A"])
suits = Set([1,2,3,4])
ranks = Set([2,3,4,5,6,7,8,9,10,11,12,13,14])
deck = cartesian_product([suits, ranks])
deck.cardinality()

52

3b. A *poker hand* consists of five distinct cards (order not important). Construct an iterator (not a list) that enumerates the possible poker hands from your deck.

In [43]:
Hands = Subsets(deck, 5)
Hands.cardinality() #matches binomial(52,5)
iterator_hands = iter(Hands)

<type 'generator'>

3c. Write a function that, given a poker hand, returns the classification of this hand according to the [Wikipedia list of poker hands](https://en.wikipedia.org/wiki/List_of_poker_hands).

In [44]:
#Helper Functions
#This function counts how many of each rank there are in the hand
#https://stackoverflow.com/questions/2161752/how-to-count-the-frequency-of-the-elements-in-a-list
def rank_counts(hand):
    from itertools import groupby
    rank_seen = sorted([b for (a,b) in hand])
    rank_seen_count = [len(list(group)) for key, group in groupby(rank_seen)]
    rank_seen = sorted(list(set(rank_seen)))
    return (rank_seen,rank_seen_count)


def is_royal_flush(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight AND all same suit
    if (rank_seen[0]==10) and (rank_seen[1]==11) and (rank_seen[2]==12) and (rank_seen[3]==13) and (rank_seen[4]==14) and len(set(a for (a,b) in hand)) == 1:
        return True
    return False


def is_straight_flush(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight AND all same suit
    if ((rank_seen[0]==2) and (rank_seen[1]==3) and (rank_seen[2]==4) and (rank_seen[3]==5) and (rank_seen[4]==14) or (rank_seen[0]+1==rank_seen[1]) and (rank_seen[1]+1==rank_seen[2]) and (rank_seen[2]+1==rank_seen[3]) and (rank_seen[3]+1==rank_seen[4])) and len(set(a for (a,b) in hand)) == 1 and (not is_royal_flush(hand)):
        return True
    return False


def is_four_of(hand):
    P = rank_counts(hand)
    #check if hand has 2 types of ranks
    if len(P[0]) != 2:
        return False
    if (1 in P[1]) and (4 in P[1]):
        return True
    else:
        return False

def is_full_house(hand):
    P = rank_counts(hand)
    #check if hand has 2 types of ranks
    if len(P[0]) != 2:
        return False
    if (2 in P[1]) and (3 in P[1]):
        return True
    else:
        return False

#https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/tutorial.html
def is_flush(hand):
    if (len(set(a for (a,b) in hand))==1) and (not is_straight_flush(hand)) and (not is_royal_flush(hand)):
        return True
    return False
#a is the suit of the card, b is the rank. Set of the suits of the cards only shows distinct suits i.e. if all suits are the same, should have length 1


def is_straight(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight
    if ((rank_seen[0]==2) and (rank_seen[1]==3) and (rank_seen[2]==4) and (rank_seen[3]==5) and (rank_seen[4]==14) or (rank_seen[0]+1==rank_seen[1]) and (rank_seen[1]+1==rank_seen[2]) and (rank_seen[2]+1==rank_seen[3]) and (rank_seen[3]+1==rank_seen[4])) and (not is_straight_flush(hand)) and (not is_royal_flush(hand)):
        return True
    return False


def is_three_of(hand):
    P = rank_counts(hand)
    #check if hand has 3 types of ranks
    if len(P[0]) != 3:
        return False
    if 3 in P[1]:
        return True
    else:
        return False


def is_two_pair(hand):
    P = rank_counts(hand)
    #check if hand has 3 types of ranks
    if len(P[0]) != 3:
        return False
    if 2 in P[1]:
        return True
    else:
        return False


def is_one_pair(hand):
    P = rank_counts(hand)
    #check if hand has 4 types of ranks
    if len(P[0]) != 4:
        return False
    if 2 in P[1]:
        return True
    else:
        return False


def is_high_card(hand):
    P = rank_counts(hand)
    #check if hand has 5 types of ranks
    if len(P[0]) != 5 or is_straight(hand) or is_flush(hand) or is_straight_flush(hand) or is_royal_flush(hand):
        return False
    else:
        return True

In [45]:
def class_hand(hand):
    if is_high_card(hand):
        return "High Card"
    elif is_one_pair(hand):
        return "1 Pair"
    elif is_two_pair(hand):
        return "2 Pair"
    elif is_three_of(hand):
        return "3 of a Kind"
    elif is_straight(hand):
        return "Straight"
    elif is_flush(hand):
        return "Flush"
    elif is_full_house(hand):
        return "Full House"
    elif is_four_of(hand):
        return "4 of a Kind"
    elif is_straight_flush(hand):
        return "Straight Flush"
    else:
        return "Royal Flush"

3d. Using your answer to 3c, compute the probability that a poker hand, chosen randomly from a deck, is of any particular type.

In [46]:
#probability vector for type of hand out of all possible hands:
#high card, 1 pair, 2 pair, 3 of a kind, straight, flush, full house, 4 of a kind, straight flush, royal flush
probs = [0 for x in range(10)]
iterator_hands = iter(Subsets(deck, 5)) #just copied from above

for i in iterator_hands:
    if class_hand(i)=="High Card":
        probs[0]+=1
    elif class_hand(i)=="1 Pair":
        probs[1]+=1
    elif class_hand(i)=="2 Pair":
        probs[2]+=1
    elif class_hand(i)=="3 of a Kind":
        probs[3]+=1
    elif class_hand(i)=="Straight":
        probs[4]+=1
    elif class_hand(i)=="Flush":
        probs[5]+=1
    elif class_hand(i)=="Full House":
        probs[6]+=1
    elif class_hand(i)=="4 of a Kind":
        probs[7]+=1
    elif class_hand(i)=="Straight Flush":
        probs[8]+=1
    else:
        probs[9]+=1

vector(probs).change_ring(RR)/binomial(52,5)

(0.501177394034537, 0.422569027611044, 0.0475390156062425, 0.0211284513805522, 0.00392464678178964, 0.00196540154523348, 0.00144057623049220, 0.000240096038415366, 0.0000138516945239634, 1.53907716932927e-6)

### Problem 4: The weak Goldbach problem

Grading criterion: correctness of code and thoroughness of analysis.

The notorious *Goldbach problem* is to prove that every even integer greater than 2 can be written as the sum of two primes. A slightly easier problem (which would follow from the original Goldbach problem) is to prove that every odd integer greater than 5 can be written as the sum of three primes. This was shown for *sufficiently large* integers by Vinogradov in 1937; the gap between "sufficiently large" and "all" was closed by Helfgott in 2013. (This involves serious use of interval arithmetic, but never mind for now.)

4a. Define a function that, given an odd positive integer $n$, returns a list of all of the ways to write $n$ as a sum of three odd primes (up to permutations) by doing a list comprehension over all partitions of $n$ into three positive integers, testing whether each summand is prime.

In [2]:
import itertools
def f1(n):
    return [t for t in itertools.combinations_with_replacement(range(1,n),3) if sum(t)==n and is_prime(t[0]) and is_prime(t[1]) and is_prime(t[2])]

4b. Define another function that does the same combination, but this time by doing a list comprehension over all combinations of three elements of the set of primes less than $n$, testing whether each triple adds up to $n$.

In [3]:
import itertools
def f2(n):
    return [t for t in itertools.combinations_with_replacement(primes(1,n),3) if sum(t)==n]

4c. To confirm that the two methods agree, count the number of solutions returned by 4a and 4b for $n=10^k+1$ for $k=1,2,3,4,5$.

In [164]:
k=1
n=10^k + 1
a = f1(n); b = f2(n)
print(len(a),len(a)==len(b))

k=2
n=10^k + 1
a = f1(n); b = f2(n)
print(len(a),len(a)==len(b))

k=3
n=10^k + 1
a = f1(n); b = f2(n)
print(len(a),len(a)==len(b))

(2, True)


(38, True)


(1095, True)


In [4]:
k=4
n=10^k + 1
a = f1(n); b = f2(n)
print(len(a),len(a)==len(b))

4d. Using the result of 4c, make a guess about how the number of solutions behaves for large $n$. You may collect additional data if that helps.

The number of solutions is approximately also the magnitude of n. For $n=10^5$, I expect the number of solutions to be of magnitude $10^5$

### Problem 5: A threshold property of random graphs

Grading criterion: correctness of code and thoroughness of analysis.

5a. Let us say that a graph is *mostly connected* if there is a single connected component containing at least 75% of the vertices. Write a function to test whether a given graph has this property.

In [26]:
def mostly_connected(G):
    H = G.connected_components_sizes()
    return H[0] >= sum(H)*.75

5b. Let $G(n, p)$ be a graph on $n$ vertices in which each possible edge is included with probability $p$. For $n = 100, 500, 1000$, plot the probability that $G(n,p)$ is mostly connected (based on a sample of 100 random graphs) as a function of $p$.

In [152]:
import matplotlib.pyplot as plt

n=100
p = [.01*x for x in range(11)]
probs = [0 for x in p]
B = 100
for j in range(len(p)):
    count = 0
    for i in range(B):
        if mostly_connected(graphs.RandomGNP(n,p[j])):
            count+=1
    probs[j] = count/B
probs
plt.plot(p,probs)
plt.axis([-.01, .1, -.05, 1.05])

[-0.0100000000000000, 0.100000000000000, -0.0500000000000000, 1.05000000000000]

In [46]:
n=500
p = [.01*x for x in range(11)]
probs = [0 for x in p]
B = 100
for j in range(len(p)):
    count = 0
    for i in range(B):
        if mostly_connected(graphs.RandomGNP(n,p[j])):
            count = count+1
    probs[j] = count/B
probs
plt.plot(p,probs)
plt.axis([-.01, .1, -.05, 1.05])

[-0.0100000000000000, 0.100000000000000, -0.0500000000000000, 1.05000000000000]

In [49]:
n=1000
p = [.01*x for x in range(11)]
probs = [0 for x in p]
B = 100
for j in range(len(p)):
    count = 0
    for i in range(B):
        if mostly_connected(graphs.RandomGNP(n,p[j])):
            count = count+1
    probs[j] = count/B
probs
plt.plot(p,probs)
plt.axis([-.01, .1, -.05, 1.05])

[-0.0100000000000000, 0.100000000000000, -0.0500000000000000, 1.05000000000000]

5c. Write a function that, for a given $n$, finds a value of $p$ for which the probability (based on a sample of 100 random graphs) that $G(n,p)$ is mostly connected is between 25% and 75%.

In [129]:
def find_p(n):
    left = 0
    right = 1
    p = (right+left)/2 #initalize at .5
    B = 100
    count = 0
    for i in range(B): #inital probability
        if mostly_connected(graphs.RandomGNP(n,p)):
            count = count + 1
    prob_mostly_connected = count/B #initial probability

    while (prob_mostly_connected < .25) or (prob_mostly_connected > .75): #while the probability is not between 25,75
        #calculating new p for this loop
        if prob_mostly_connected < .25: #takes midpoint of left interval
            left = p
            p = (right+left)/2
        if prob_mostly_connected > .75: #takes midpoint of right interval
            right = p
            p = (right+left)/2

        #calculating the probability that G(n,p) is mostly connected given p
        count = 0
        for i in range(B):
            if mostly_connected(graphs.RandomGNP(n,p)):
                count = count + 1
        prob_mostly_connected = count/B
        #print(N(prob_mostly_connected),p) #testing

    return p

In [131]:
# left = 0
# right = 1
# p = (right+left)/2 #initalize at .5
# prob_mostly_connected = .8

# if prob_mostly_connected < .25: #takes midpoint of left interval
#     left = p
#     p = (right+left)/2
# if prob_mostly_connected > .75: #takes midpoint of right interval
#     right = p
#     p = (right+left)/2
# print(left,right,N(p))

print(find_p(100))
#find_p_test(10)
# H = graphs.RandomGNP(10,.5)
# show(H)
# mostly_connected(H)

9/512


5d. Plot the values of $p$ you computed in (b) as a function of $n$, going up to at least $n=1000$ with at least 10 sample points. Then make a guess as to how these values depend on $n$.

In [150]:
import matplotlib.pyplot as plt

x = [50,100] + [100*i for i in range(2,11)]; x
p = [find_p(i) for i in x];
plt.plot(x,p)

### Problem 6: Spectral gaps of graphs

Grading criterion: correctness of code and thoroughness of analysis.

6a. Let $G$ be a graph on $n$ vertices which is $k$-regular (each vertex has $k$ neighbors). Let $A$ be the adjacency matrix of $G$. Explain what the Perron-Frobenius theorem implies about the eigenvalues of $A$.

Souce: https://en.wikipedia.org/wiki/Perron%E2%80%93Frobenius_theorem

The Perron-Frobenius theorem says that for any real valued square matrix with positive entries has a unique largest real eigenvalue. Thus A, the adjacency matrix of G, has a unique largest real eigenvalue that is bounded above

6b. Let $G_n$ be the graph with vertices $\{0,\dots,n-1\}$ in which $x$ is adjacent to $y$ if and only if $x - y$ is congruent to one of $\pm 1, \dots, \pm k$ modulo $n$. Write a function that constructs the adjacency matrix of $G_n$ and returns the norm of its second-largest eigenvalue.

In [74]:
def norm_eig1(n,k):
    #constructs the adjacency matrix
    A = matrix(n)
    congruences = [x%n for x in range(1,k+1)] + [-x%n for x in range(1,k+1)]
    for i in range(n):
        for j in range(n):
            if (i-j)%n in congruences:
                A[i,j] = 1

    e = np.linalg.eig(A)[0]
    #print(np.sort(e))
    return abs(np.sort(e)[len(e)-2])

In [75]:
n=100
k=3
print(norm_eig1(n,k))
print('---')
print(norm_eig2(n,k))

#norm_eig1(n,k) == norm_eig2(n,k)
#np.sort([-3-1j,-1+2j,1-1j,1+1j]) #no way to order complex #s

5.94485736094
---
4.13109476981


6c. Write a function that constructs the adjacency matrix of a random $2k$-regular graph on $n$ vertices and returns the norm of its second-largest eigenvalue.

In [73]:
def norm_eig2(n,k):
    import numpy as np
    Adj = graphs.RandomRegular(2*k, n).adjacency_matrix()
    e = np.linalg.eig(Adj)[0]
    #print(np.sort(e))
    return abs(np.sort(e)[len(e)-2])

6d. For $k=3,4,5$, make a plot that compares the answers of 6b and 6c for $n$ from 100 to 1000 (with at least 10 sample points).

In [81]:
k=3
import matplotlib.pyplot as plt

n = [100*i for i in range(1,11)]
e1 = [norm_eig1(i,k) for i in n]
e2 = [norm_eig2(i,k) for i in n]
plt.plot(n,e1,'b-',linewidth=3)
plt.plot(n,e2,'r-',linewidth=3)

In [82]:
k=4
import matplotlib.pyplot as plt

n = [100*i for i in range(1,11)]
e1 = [norm_eig1(i,k) for i in n]
e2 = [norm_eig2(i,k) for i in n]
plt.plot(n,e1,'b-',linewidth=3)
plt.plot(n,e2,'r-',linewidth=3)

In [83]:
k=5
import matplotlib.pyplot as plt

n = [100*i for i in range(1,11)]
e1 = [norm_eig1(i,k) for i in n]
e2 = [norm_eig2(i,k) for i in n]
plt.plot(n,e1,'b-',linewidth=3)
plt.plot(n,e2,'r-',linewidth=3)

6e. In as much detail as possible, interpret your answer to 6d in terms of mixing of random walks.

Source: http://web.cs.elte.hu/~lovasz/eigenvals-x.pdf

The gap between norms of the second biggest eigenvalue is useful as a measure of connectivity of a graph. Related to the measure of connectivity is the bound of the mixing time of random walks on a graph. 